# pytorchvideo UFC101, pytorchvideo X3D pretrain/scratch

pytorchvideonのdatasetを使ってUFC101を読み込み，pytorchvideoのx3dモデルをfine-tuningしてみる．
UFC101はあらかじめダウンロードして展開済みであるとする．

- https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#ucf101

- https://pytorch.org/hub/facebookresearch_pytorchvideo_x3d/



## ダウンロードできないというエラー

torchvisionをimportした後ではエラーが発生する（ImportError: cannot import name ***）

- https://github.com/pytorch/hub/issues/46


## 対応策

import torch直後に（import torchvisionをしない状態で）torch.hub.loadして，キャッシュに残しておく

こうすると，以降はキャッシュ（~/.cache/torch/hub/checkpoints/）が使われるのでエラーは発生しない

In [15]:

import torch
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True)
model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)


Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master
Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master
Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DistributedSampler, RandomSampler


from torchvision import transforms


from pytorchvideo.models import x3d
from pytorchvideo.data import Ucf101, RandomClipSampler, UniformClipSampler


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)


#import torchinfo

from tqdm.notebook import tqdm
import itertools
import os
import pickle

argparseを真似たパラメータ設定．
- rootで指定したディレクトリには，101クラスのサブディレクトリがあること
- annotation_pathにはtrainlist0{1,2,3}.txtなどがあること

In [17]:
class Args:
    def __init__(self):
        self.metadata_path = '/mnt/NAS-TVS872XT/dataset/UFC101/'
        self.root = self.metadata_path + 'video/'
        self.annotation_path = self.metadata_path + 'ucfTrainTestlist/'
        self.frames_per_clip = 16
        self.step_between_clips = 16
        self.model = 'x3d_m'
        self.batch_size = 16
        self.num_workers = 24

        self.clip_duration = 16/25  # 25FPSを想定して16枚
        self.video_num_subsampled = 16  # 16枚抜き出す

args = Args()

transformの定義．
- UniformTemporalSubsampleで固定枚数をサンプルする
 - datasetのclip_samplerには，秒単位でしか与えられないようなので，fpsが異なる動画ではサンプルされる枚数も変わってくる．そのためここで取得するフレーム数を揃える（もっといい方法はないのか？）
- UCF101を読み込むとfloat32だが値は0-255，255で割ってfloatにする．
- X3D-Mを想定して，短い方を256画素程度に合わせてから，画像を224x224にリサイズする．
  - RandomShortSideScaleなら厳密には256にならない
  - ShortSideScaleなら256になる

バッチはdict形式なので，video, label, audioなどのそれぞれにtransformが設定できる
- ApplyTransformToKeyでkeyを指定して，video用のtransformを設定
- UCF101のラベルファイル（trainlist01.txtなど）には1から101までのラベルが付いているが，それがそのまま使われてしまうので（なぜだ．．．），このままではエラーが（不定期に）発生する．ラベルの値をtransformでから100にしておく
- audioは使わないのでRemoveKeyで除去

In [18]:
train_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ## 以下デバッグ用
                # transforms.Lambda(lambda x: [
                #     x, 
                #     print(type(x)),
                #     print(x.dtype),
                #     print(x.max()),
                #     print(x.min()),
                #     print(x.mean()),
                #     ]),
                # transforms.Lambda(lambda x: x[0]),
                RandomShortSideScale(min_size=256, max_size=320,),
                RandomCrop(224),
                RandomHorizontalFlip(),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])

val_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ShortSideScale(256),
                CenterCrop(224),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])



In [21]:
root_UCF101 = '/mnt/NAS-TVS872XT/dataset/UCF101/'

train_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/trainlist01.txt',  # ラベルが1から101になっているので，transformで1を引いている
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=args.clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=train_transform,
    )
val_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/testlist01.txt',
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=args.clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=val_transform,
    )

num_classes = 101

In [22]:
train_set.num_videos

9537

In [23]:
val_set.num_videos

3783

In [24]:
# https://github.com/facebookresearch/pytorchvideo/blob/ef2d3a96bb939b12aa0f21fb467d2175b0f05e9f/tutorials/video_classification_example/train.py#L343
class LimitDataset(torch.utils.data.Dataset):
    """
    To ensure a constant number of samples are retrieved from the dataset we use this
    LimitDataset wrapper. This is necessary because several of the underlying videos
    may be corrupted while fetching or decoding, however, we always want the same
    number of steps per epoch.
    """

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

In [25]:
train_loader = DataLoader(LimitDataset(train_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)
val_loader = DataLoader(LimitDataset(val_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)


/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:475: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


データローダのlenを確認．
- trainlist01.txtには9537行あるので「サンプル数＝ビデオ数」
- バッチサイズで割るとtrain_loaderのlengthになる

In [26]:
len(train_loader), train_set.num_videos, train_set.num_videos / args.batch_size

(596, 9537, 596.0625)

data loaderの挙動を確認．
- バッチはdictでやってくるので，`batch['video']`と`batch['label']`で取り出す
- RandomClipSamplerならランダムなラベルが得られている．

In [27]:
for i, batch in enumerate(train_loader):
    if i == 0:
        print(batch.keys())
        print(batch['video'].shape)
    print(batch['label'].cpu().numpy())
    if i > 10:
        break

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
torch.Size([16, 3, 16, 224, 224])
[79 58 92 52  3 43 62 82 53 60 10 31 91 52  5  6]
[ 4 12 69 82  4 95 50 80 49  9 80 70 10 64 70 46]
[95 91  0 52 19 57 77 23 23 86 69  3 17 78 34 83]
[90  5 72 48 85 81 45 53 94 54 94 42 36 98 41 78]
[ 6 81 91 87 18 23 11  4 72 21 52 25 69 24 25 78]
[13 84 93 87 12 41 66 16 61 72 24 73 28 17 19 64]
[14 33 64 15 13 46 73 19 14 84 40 75 37 40 97 96]
[41 33 86 68 54 89 67 45 11 86 16  3 65 38 64 59]
[18 26 98 98  8 21 35 34 66  1  3 96 82 71 53 84]
[43 26 34 92 41 33 20 10 84 70 52 21 24 82 31 36]
[26 59 64 53 92 77 12 13 34 58 60  0 51 37 39 88]
[41 54 41 78 12 26  8 96 14 81 37 66 50 55 12 46]


In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

pytorchvideoのpretrained x3dモデルをダウンロード．
あとでsummaryを見れば分かるように，最終線形層は`model.blocks[5].proj`だからこれをnn.Linearに置き換える

- 注意：エラーが発生してダウンロードできない場合には，このnotebookの冒頭の注意書きを確認すること

In [29]:
# # X3D-M
# # https://github.com/facebookresearch/pytorchvideo/blob/master/pytorchvideo/models/x3d.py#L601
# model = x3d.create_x3d(
#     input_clip_length=16,
#     input_crop_size=224,
#     depth_factor=2.2,
#     model_num_class=101
# ).to(device)


model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True)

# fine-tuningするなら以下を実行．スクラッチで学習するなら，実行しない
do_fine_tune = True
if do_fine_tune:
    for param in model.parameters():
        param.requires_grad = False

model.blocks[5].proj = nn.Linear(model.blocks[5].proj.in_features, num_classes)
model = model.to(device)

# data parallelだと性能が落ちる（設定次第？）
# model = nn.DataParallel(model)

Using cache found in /home/omi/.cache/torch/hub/facebookresearch_pytorchvideo_master


ランダムなデータを流し込んで出力されるかを確認する

In [30]:
data = torch.randn(2, 3, 16, 224, 224).to(device)

In [31]:
model(data)

tensor([[0.0112, 0.0102, 0.0098, 0.0099, 0.0096, 0.0096, 0.0090, 0.0101, 0.0093,
         0.0108, 0.0096, 0.0086, 0.0101, 0.0103, 0.0109, 0.0096, 0.0094, 0.0095,
         0.0100, 0.0093, 0.0096, 0.0097, 0.0110, 0.0092, 0.0114, 0.0088, 0.0103,
         0.0110, 0.0097, 0.0096, 0.0107, 0.0088, 0.0087, 0.0104, 0.0105, 0.0117,
         0.0096, 0.0090, 0.0101, 0.0104, 0.0091, 0.0105, 0.0104, 0.0093, 0.0101,
         0.0099, 0.0114, 0.0109, 0.0095, 0.0106, 0.0086, 0.0112, 0.0098, 0.0101,
         0.0090, 0.0092, 0.0102, 0.0096, 0.0105, 0.0117, 0.0104, 0.0102, 0.0089,
         0.0093, 0.0109, 0.0095, 0.0111, 0.0084, 0.0085, 0.0117, 0.0110, 0.0090,
         0.0090, 0.0101, 0.0106, 0.0093, 0.0092, 0.0095, 0.0086, 0.0106, 0.0106,
         0.0092, 0.0099, 0.0111, 0.0089, 0.0085, 0.0094, 0.0102, 0.0108, 0.0103,
         0.0097, 0.0109, 0.0094, 0.0087, 0.0099, 0.0101, 0.0090, 0.0101, 0.0111,
         0.0101, 0.0078],
        [0.0087, 0.0098, 0.0103, 0.0110, 0.0089, 0.0088, 0.0101, 0.0090, 0.0093,
  

summaryで中身を確認

In [33]:
# torchinfo.summary(
#     model,
#     (4, 3, 16, 224, 224),
#     depth=4,
#     col_names=["input_size",
#                "output_size"],
#     row_settings=("var_names",)
# )

IndentationError: unexpected indent (<ipython-input-33-a91382979121>, line 2)

便利関数を定義

In [17]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    https://github.com/machine-perception-robotics-group/attention_branch_network/blob/ced1d97303792ac6d56442571d71bb0572b3efd8/utils/misc.py#L59
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if type(val) == torch.Tensor:
            val = val.item()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def top1(outputs, targets):
    batch_size = outputs.size(0)
    _, predicted = outputs.max(1)
    return predicted.eq(targets).sum().item() / batch_size

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [20]:
num_epochs = 5

model = model.to(device)

with tqdm(range(num_epochs)) as pbar_epoch:
    for epoch in pbar_epoch:
        pbar_epoch.set_description("[Epoch %d]" % (epoch))


        with tqdm(enumerate(train_loader),
                  total=len(train_loader),
                  leave=True) as pbar_loss:

            train_loss = AverageMeter()
            train_acc = AverageMeter()
            model.train()

            for batch_idx, batch in pbar_loss:
                pbar_loss.set_description("[train]")

                inputs, targets = batch['video'].to(device), batch['label'].to(device)
                bs = inputs.size(0)  # current batch size, may vary at the end of the epoch

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss.update(loss, bs)
                train_acc.update(top1(outputs, targets), bs)

                pbar_loss.set_postfix_str(
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ''.format(
                    train_loss.avg, train_acc.avg,
                    train_loss.val, train_acc.val,
                ))



fine-tuningなので速い．
- 4GPUでおよそ4.5it/s，1エポック約2分
- 1GPUでおよそ5it/s，1エポック約3分（596 iterations）

スクラッチで学習するなら
- 4GPUでおよそ2.6it/s，1エポック約4分
- 1GPUでおよそ1.8it/s，1エポック約5.5分（596 iterations）


以下の設定
- video_num_subsampled = 16
- batch_size = 16
- num_workers = 24